# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [184]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [185]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [186]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [188]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [247]:
root = ET.parse('./data/mondial_database.xml')

In [260]:
countryList = []
moralityList = []
for element in root.iterfind('country'):
        mortality = element.find('infant_mortality')
        countryName = element.find('name').text
        mortalityVal = 0.0 
        if(mortality is not None):
            mortalityVal = float(mortality.text)
            countryList.append(countryName)
            moralityList.append(mortalityVal)
#print countryList
dict = {'country':countryList,'mortality':moralityList}
df = pd.DataFrame(dict, columns=('country', 'mortality')).sort_values(by='mortality')
df.head(10)

,country,mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [261]:
#Top Ten populated cities with in a year, where year is a parameter
def getTopTenPopulatedCitiesByYear(root,year):
    countryList = []
    cityList = []
    yearList = []
    populationList = []
    
    for element in root.iterfind('country'):
            for city in element.iterfind('city'):
                cityName = city.find('name').text
                for population in city.iterfind('population'):
                    cityList.append(cityName)
                    #print population.attrib['year']
                    populationList.append(np.int(population.text))
                    yearList.append(population.attrib['year'])

    #print countryList
    dict = {'city':cityList,'population':populationList,'year':yearList}
    df = pd.DataFrame(dict, columns=('city', 'year','population'))
    df = df[df['year'] == year][['city','population']]
    return df.sort_values(by='population',ascending=False).head(10)


#You can vary the year value below to get top 10 most populated cities for that year
getTopTenPopulatedCitiesByYear(root,'2012')

,city,population
566,New Taipei,3939305
604,Kaohsiung,2778659
608,Taichung,2684893
563,Taipei,2673226
612,Tainan,1881645
910,Harare,1485231
642,Managua,1028808
630,Ciudad de Guatemala,992541
913,Bulawayo,653337
636,Villa Nueva,527174


In [296]:
def getEthnicDF(root):
    countryNameList = []
    ethnicGroupNameList = []
    ethnicGroupPopulationList = []
    totalPopulationList = []
    for element in root.iterfind('country'):
        countryName =  element.find('name').text
        latestYearPopulation = element.findall('population')[-1].text
        for ethnicGroup in element.iterfind('ethnicgroup'):
            countryNameList.append(countryName)
            ethnicGroupName = ethnicGroup.text
            ethnicGroupNameList.append(ethnicGroupName)
            totalPopulationList.append(latestYearPopulation)
            egp = ethnicGroup.get('percentage')
            ethnicGroupPopulation = (np.float(egp)*np.float(latestYearPopulation))/100
            ethnicGroupPopulationList.append(ethnicGroupPopulation)
            
    dict = {'countryName':countryNameList,'ethnicGroupName':ethnicGroupNameList,'ethnicGroupPopulation':ethnicGroupPopulationList,'totalPopulation':totalPopulationList}
    ethnicDF = pd.DataFrame(dict, columns=('countryName', 'ethnicGroupName','ethnicGroupPopulation','totalPopulation'))
    return ethnicDF

ethnicDF = getEthnicDF(root)
#ethnicDF
ethnicDF.groupby(['ethnicGroupName'])['ethnicGroupPopulation'].sum().sort_values(ascending=False).head(10)

ethnicGroupName
Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.318570e+08
Japanese       1.265342e+08
Malay          1.219936e+08
Name: ethnicGroupPopulation, dtype: float64

In [245]:
#Common Function
def getCountryDF(root):
    countryCodeList = []
    countryNameList = []
    riverList = []
    for element in root.iterfind('country'):
            car_code = element.get('car_code')
            countryName = element.find('name').text
            #if(car_code is not None):
            countryCodeList.append(car_code)
            countryNameList.append(countryName)
    #print countryList
    dict = {'countryCode':countryCodeList,'countryName':countryNameList}
    cdf = pd.DataFrame(dict, columns=('countryCode', 'countryName'))
    return cdf


In [250]:
#Longest River
def getRiverDF(root):
    countryCodeList = []
    riverNameList = []
    riverLenList = []
    for element in root.iterfind('river'):
            source =  element.find('source')
            srcCode = source.get('country')
            length = element.find('length')
            if(length is not None):
                riverLength = np.float(length.text)
                riverName = element.find('name').text
                countryCodeList.append(srcCode)
                riverNameList.append(riverName)
                riverLenList.append(riverLength)
    #print countryList
    dict = {'countryCode':countryCodeList,'riverName':riverNameList,'riverLength':riverLenList}
    riverDF = pd.DataFrame(dict, columns=('countryCode', 'riverName','riverLength'))
    return riverDF

countryDF = getCountryDF(root)
riverDF = getRiverDF(root)

f = pd.merge(riverDF, countryDF, how='inner', left_on=['countryCode'], right_on=['countryCode'])

f.sort_values(by="riverLength", ascending=False).head(1)


,countryCode,riverName,riverLength,countryName
171,PE,Amazonas,6448,Peru


In [273]:
#Largest Lake
def getLakeDF(root):
    countryCodeList = []
    lakeNameList = []
    lakeLenList = []
    for element in root.iterfind('lake'):
            srcCode =  element.get('country').split()[0]
            area = element.find('area')
            if(area is not None):
                lakeLength = np.float(area.text)
                lakeName = element.find('name').text
                countryCodeList.append(srcCode)
                lakeNameList.append(lakeName)
                lakeLenList.append(lakeLength)
    #print countryList
    dict = {'countryCode':countryCodeList,'lakeName':lakeNameList,'lakeArea':lakeLenList}
    lakeDF = pd.DataFrame(dict, columns=('countryCode', 'lakeName','lakeArea'))
    return lakeDF
    
countryDF = getCountryDF(root)
lakeDF = getLakeDF(root)

lkf = pd.merge(lakeDF, countryDF, how='inner', left_on=['countryCode'], right_on=['countryCode'])

lkf.sort_values(by="lakeArea", ascending=False).head(1)


,countryCode,lakeName,lakeArea,countryName
47,R,Caspian Sea,386400,Russia


In [259]:
#Aiport with highest elevation
def getAirportDF(root):
    countryCodeList = []
    airportNameList = []
    airportElevationList = []
    for element in root.iterfind('airport'):
            srcCode =  element.get('country')
            elevation = element.find('elevation')
            if(elevation is not None):
                if(elevation.text is not None):
                    airportElevation = np.float(elevation.text)
                    airportName = element.find('name').text
                    countryCodeList.append(srcCode)
                    airportNameList.append(airportName)
                    airportElevationList.append(airportElevation)
    #print countryList
    dict = {'countryCode':countryCodeList,'airportName':airportNameList,'airportElevation':airportElevationList}
    airportDF = pd.DataFrame(dict, columns=('countryCode', 'airportName','airportElevation'))
    return airportDF
    
countryDF = getCountryDF(root)
airportDF = getAirportDF(root)

lkf = pd.merge(airportDF, countryDF, how='inner', left_on=['countryCode'], right_on=['countryCode'])

lkf.sort_values(by="airportElevation", ascending=False).head(1)

,countryCode,airportName,airportElevation,countryName
80,BOL,El Alto Intl,4063,Bolivia
